In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

train_csv = "/Users/hunjunsin/Desktop/Jun/Unsupervised/hw1/20news_train.csv"
test_csv = "/Users/hunjunsin/Desktop/Jun/Unsupervised/hw1/20news_test.csv"

train_data = pd.read_csv(train_csv)
test_data = pd.read_csv(test_csv)

combined_data = pd.concat([train_data, test_data], ignore_index=True)

combined_data = combined_data.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"data number : {len(combined_data)}")


data number : 18846


In [22]:
sample = combined_data.sample(n=9000, random_state=42)

In [23]:
len(sample)

9000

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

data_20 = sample.dropna(subset = ['body']).reset_index(drop = True)
train_documents = data_20['body'].reset_index(drop=True)
train_labels = data_20['label'].reset_index(drop=True)

In [25]:
train_documents

0       starr@kuhub.cc.ukans.edu writes:\n: I need hel...
1       In article <1993Apr15.003015.1@vmsb.is.csupomo...
2       In article <1993Apr25.151108.1@aurora.alaska.e...
3       In article <93059@hydra.gatech.EDU> gt4356c@pr...
4       Kevin Kimmell - Computer Science/German Underg...
                              ...                        
8984    In article <C5t14M.Ku2@acsu.buffalo.edu> v111q...
8985    NNTP-Posting-Host: po2.andrew.cmu.edu\nIn-Repl...
8986    In article <sandvik-160493205355@sandvik-kent....
8987    In article <Stafford-210493125617@stafford.win...
8988    In <16BB91429.C445585@mizzou1.missouri.edu> C4...
Name: body, Length: 8989, dtype: object

In [27]:
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import spacy
import nltk
from nltk.corpus import stopwords
import re
import matplotlib.pyplot as plt

nltk.download('stopwords')
stop_words = stopwords.words('english')

def reg_data(text):
    text = str(text)  # Convert to string if not already
    text = re.sub('\\s+', ' ', text)  
    text = re.sub('\\S*@\\S*\\s?', '', text)  
    text = re.sub('\'', '', text)  
    text = re.sub('[^a-zA-Z]', ' ', text)  
    text = text.lower()  
    return text

# Tokenize and remove stopwords
def tok_data(data):
    tokens = gensim.utils.simple_preprocess(data, deacc=True)
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

# Lemmatization using spaCy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
def lem_data(tokens):
    doc = nlp(" ".join(tokens))
    return [token.lemma_ for token in doc]

def preprocess_document(text):
    cleaned_text = reg_data(text)
    tokens = tok_data(cleaned_text)
    lemmatized_tokens = lem_data(tokens)
    return lemmatized_tokens
    
final_document = train_documents.apply(preprocess_document)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hunjunsin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/Users/hunjunsin/.pyenv/versions/3.11.6/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [28]:
final_document

0       [write, need, help, zx, put, supertrapp, slip,...
1       [article, write, today, cub, activate, mike, h...
2       [article, write, like, option, new, space, sta...
3       [article, james, dean, barwick, write, permene...
4       [kevin, kimmell, computer, science, german, un...
                              ...                        
8984    [article, vasilion, write, well, still, land, ...
8985    [nntp, post, host, po, andrew, cmu, edu, reply...
8986    [article, kent, sandvik, write, stephen, write...
8987    [article, john, stafford, write, article, bill...
8988    [john, kelsey, write, I, ve, recently, read, p...
Name: body, Length: 8989, dtype: object

In [32]:
# Create dictionary and corpus
id_word = corpora.Dictionary(final_document)
corpus = [id_word.doc2bow(doc) for doc in final_document]

K = 10

In [36]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id_word,
                                            num_topics=10, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

In [40]:
# Print the topics
topics = lda_model.print_topics(num_words=20)
for topic in topics:
    print(topic)

(0, '0.010*"god" + 0.009*"people" + 0.007*"say" + 0.006*"believe" + 0.005*"one" + 0.005*"man" + 0.004*"we" + 0.004*"government" + 0.004*"state" + 0.004*"law" + 0.004*"right" + 0.004*"may" + 0.004*"christian" + 0.004*"word" + 0.004*"mean" + 0.004*"issue" + 0.003*"fact" + 0.003*"evidence" + 0.003*"world" + 0.003*"child"')
(1, '0.029*"game" + 0.017*"play" + 0.016*"team" + 0.016*"gun" + 0.011*"win" + 0.010*"weapon" + 0.010*"period" + 0.009*"buf" + 0.008*"st" + 0.007*"hockey" + 0.007*"goal" + 0.006*"pp" + 0.006*"la" + 0.006*"san" + 0.006*"pt" + 0.006*"packet" + 0.006*"firearm" + 0.006*"scorer" + 0.005*"season" + 0.005*"score"')
(2, '0.466*"ax" + 0.041*"max" + 0.008*"pl" + 0.006*"tl" + 0.005*"hz" + 0.005*"ww" + 0.004*"di" + 0.004*"wc" + 0.004*"qs" + 0.004*"pu" + 0.004*"tm" + 0.004*"mf" + 0.004*"ah" + 0.003*"mw" + 0.003*"mg" + 0.003*"ei" + 0.003*"tiff" + 0.003*"wt" + 0.003*"fi" + 0.003*"vf"')
(3, '0.041*"jesus" + 0.020*"col" + 0.012*"christ" + 0.010*"temple" + 0.010*"telnet" + 0.008*"testamen

K = 20

In [47]:
# Build LDA model
lda_model_20 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id_word,
                                            num_topics=20, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

In [49]:
# Print the topics
topics_20 = lda_model_20.print_topics(num_words=20)
for topic in topics_20:
    print(topic)

(0, '0.014*"people" + 0.012*"one" + 0.011*"say" + 0.010*"may" + 0.008*"mean" + 0.008*"make" + 0.007*"man" + 0.007*"many" + 0.007*"question" + 0.007*"word" + 0.007*"fact" + 0.006*"believe" + 0.006*"world" + 0.006*"must" + 0.006*"follow" + 0.005*"group" + 0.005*"issue" + 0.005*"would" + 0.005*"also" + 0.005*"claim"')
(1, '0.060*"government" + 0.042*"gun" + 0.041*"law" + 0.028*"right" + 0.026*"weapon" + 0.022*"state" + 0.016*"protect" + 0.016*"freedom" + 0.015*"control" + 0.015*"clinton" + 0.015*"switch" + 0.015*"legal" + 0.014*"arm" + 0.014*"koresh" + 0.013*"evidence" + 0.013*"public" + 0.012*"federal" + 0.012*"carry" + 0.011*"batf" + 0.011*"burn"')
(2, '0.135*"game" + 0.078*"play" + 0.075*"team" + 0.059*"player" + 0.026*"scorer" + 0.025*"score" + 0.024*"goal" + 0.019*"van" + 0.017*"playoff" + 0.017*"boston" + 0.014*"cup" + 0.012*"devil" + 0.011*"keith" + 0.011*"ranger" + 0.011*"montreal" + 0.011*"greg" + 0.009*"quebec" + 0.008*"selanne" + 0.008*"final" + 0.008*"winnipeg"')
(3, '0.000*"g

The 20 topics generally align well with 20NG classes, clearly capturing major themes like religion, sports, and computing.
Increasing to 50 topics allows for more fine-grained distinctions but may introduce redundancy or overly narrow topics.
Thus, increasing topic count offers better alignment but also brings the risk of added noise.

K=50

In [50]:
lda_model_50 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id_word,
                                            num_topics=50, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

In [51]:
topics_50= lda_model_50.print_topics(num_topics=50, num_words=10)
for topic in topics_50:
    print(topic)

(0, '0.081*"field" + 0.063*"object" + 0.059*"society" + 0.039*"sex" + 0.038*"stat" + 0.036*"associate" + 0.034*"beyond" + 0.030*"ice" + 0.027*"association" + 0.025*"south"')
(1, '0.000*"fribourg" + 0.000*"flint" + 0.000*"gonchar" + 0.000*"gemeinschaft" + 0.000*"gasser" + 0.000*"frolunda" + 0.000*"gordie" + 0.000*"francais" + 0.000*"finals" + 0.000*"flair"')
(2, '0.053*"suit" + 0.035*"football" + 0.029*"ref" + 0.003*"ou" + 0.000*"frolunda" + 0.000*"gonchar" + 0.000*"gemeinschaft" + 0.000*"gordie" + 0.000*"gasser" + 0.000*"flair"')
(3, '0.000*"fribourg" + 0.000*"flint" + 0.000*"gonchar" + 0.000*"gemeinschaft" + 0.000*"gasser" + 0.000*"frolunda" + 0.000*"gordie" + 0.000*"francais" + 0.000*"finals" + 0.000*"flair"')
(4, '0.120*"entry" + 0.076*"win" + 0.037*"rule" + 0.029*"pick" + 0.026*"throw" + 0.026*"watch" + 0.024*"total" + 0.020*"decision" + 0.019*"round" + 0.019*"thomas"')
(5, '0.086*"door" + 0.083*"shall" + 0.067*"european" + 0.045*"prayer" + 0.043*"lord" + 0.042*"clean" + 0.033*"fat

NMF

In [45]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer(
    max_features=10000,
    lowercase=True,
    stop_words='english',
    max_df=0.8,
    min_df=5
)

processed_texts = [' '.join(doc) for doc in final_document]
tfidf_matrix = vectorizer.fit_transform(processed_texts)

feature_names = vectorizer.get_feature_names_out()

def print_topics(model, feature_names, n_top_words):
    topics = []
    for topic_idx, topic in enumerate(model.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1]]
        topics.append((topic_idx, ' + '.join(top_words)))
    return topics

n_topics_list = [10, 20, 50]
for n_topics in n_topics_list:
    print(f"\nNMF with {n_topics} topics:")
    print("-" * 50)
    
    nmf_model = NMF(
        n_components=n_topics,
        random_state=42,
        init='nndsvd',
        max_iter=200
    )
    nmf_output = nmf_model.fit_transform(tfidf_matrix)
    
    topics = print_topics(nmf_model, feature_names, 10)
    for topic_idx, topic in topics:
        print(f"Topic {topic_idx}: {topic}")


NMF with 10 topics:
--------------------------------------------------
Topic 0: car + bike + engine + buy + dod + ride + good + like + price + oil
Topic 1: window + file + program + run + use + font + application + manager + ms + windows
Topic 2: game + team + play + player + hockey + win + year + fan + baseball + season
Topic 3: god + jesus + christian + believe + bible + say + christ + faith + sin + atheist
Topic 4: drive + scsi + disk + ide + hard + mb + controller + floppy + meg + boot
Topic 5: key + chip + clipper + encryption + government + use + phone + escrow + algorithm + nsa
Topic 6: israel + armenian + jews + arab + israeli + turkish + muslim + kill + jewish + war
Topic 7: thank + mail + email + know + information + advance + send + list + address + post
Topic 8: card + monitor + driver + video + color + mhz + bus + mb + bit + port
Topic 9: people + write + gun + article + think + say + right + make + like + government

NMF with 20 topics:
----------------------------------

/Users/hunjunsin/.pyenv/versions/3.11.6/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1741: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


NMF with 10 topics:
--------------------------------------------------
Topic 0: car + bike + engine + buy + dod + ride + good + like + price + oil

Topic 1: window + file + program + run + use + font + application + manager + ms + windows

Topic 2: game + team + play + player + hockey + win + year + fan + baseball + season

Topic 3: god + jesus + christian + believe + bible + say + christ + faith + sin + atheist

Topic 4: drive + scsi + disk + ide + hard + mb + controller + floppy + meg + boot

Topic 5: key + chip + clipper + encryption + government + use + phone + escrow + algorithm + nsa

Topic 6: israel + armenian + jews + arab + israeli + turkish + muslim + kill + jewish + war

Topic 7: thank + mail + email + know + information + advance + send + list + address + post

Topic 8: card + monitor + driver + video + color + mhz + bus + mb + bit + port

Topic 9: people + write + gun + article + think + say + right + make + like + government

NMF with 20 topics:
--------------------------------------------------
Topic 0: write + think + article + say + people + know + like + thing + make + read

Topic 1: window + run + font + application + os + ms + manager + windows + program + use

Topic 2: team + player + play + year + win + season + fan + good + hockey + hit

Topic 3: god + jesus + believe + christian + bible + christ + sin + faith + love + atheist

Topic 4: drive + scsi + disk + ide + hard + controller + mb + floppy + meg + boot

Topic 5: key + chip + clipper + encryption + phone + government + escrow + algorithm + use + nsa

Topic 6: fbi + batf + government + koresh + child + compound + clinton + people + waco + warrant

Topic 7: sale + offer + price + sell + new + good + book + condition + include + interested

Topic 8: card + driver + video + bus + color + mb + graphic + vlb + diamond + dx

Topic 9: armenian + turkish + muslim + armenians + armenia + genocide + turk + soviet + greek + turkey

Topic 10: car + engine + oil + dealer + model + ford + mile + speed + auto + look

Topic 11: thank + mail + advance + know + email + appreciate + hi + information + address + reply

Topic 12: file + program + image + format + ftp + gif + convert + graphic + directory + disk

Topic 13: homosexual + cramer + gay + man + clayton + sexual + sex + promiscuous + male + homosexuality

Topic 14: israel + arab + israeli + jews + jewish + kill + peace + palestinians + state + israelis

Topic 15: bike + dod + ride + motorcycle + write + article + dog + ve + bmw + road

Topic 16: game + espn + baseball + hockey + playoff + night + pen + cap + score + penguin

Topic 17: gun + crime + weapon + firearm + law + criminal + control + people + right + police

Topic 18: use + monitor + problem + modem + mac + work + computer + port + apple + switch

Topic 19: space + launch + shuttle + orbit + nasa + cost + satellite + science + earth + moon

## NMF with 50 topics:

---

**Topic 0:**  
people + think + like + thing + want + good + make + ve + time + look  

**Topic 1:**  
window + os + ms + manager + run + windows + application + microsoft + app + crash  

**Topic 2:**  
win + new + boston + st + vs + bruin + period + cup + montreal + blue  

**Topic 3:**  
god + believe + love + atheist + hell + sin + exist + existence + create + bible  

**Topic 4:**  
drive + hard + floppy + ide + boot + meg + controller + format + mb + jumper  

**Topic 5:**  
key + bit + pgp + encrypt + escrow + des + public + message + security + attack  

**Topic 6:**  
driver + version + ftp + diamond + site + mode + ati + available + video + vesa  

**Topic 7:**  
thank + advance + hi + appreciate + help + email + information + info + look + anybody  

**Topic 8:**  
card + video + bus + graphic + controller + vlb + slot + color + isa + pc  

**Topic 9:**  
gun + crime + firearm + weapon + criminal + control + handgun + police + carry + rate  

**Topic 10:**  
car + engine + oil + model + dealer + ford + mile + speed + auto + buy  

**Topic 11:**  
bike + dod + ride + motorcycle + bmw + dog + rider + road + buy + ve  

**Topic 12:**  
file + format + directory + convert + ftp + gif + bmp + exe + zip + swap  

**Topic 13:**  
homosexual + cramer + gay + man + clayton + sexual + sex + promiscuous + male + homosexuality  

**Topic 14:**  
israel + arab + israeli + jews + jewish + kill + palestinians + peace + israelis + soldier  

**Topic 15:**  
armenian + turkish + armenians + armenia + turk + genocide + soviet + turkey + greek + massacre  

**Topic 16:**  
game + espn + baseball + hockey + night + playoff + pen + fan + cap + play  

**Topic 17:**  
team + player + play + hockey + fan + nhl + season + year + good + league  

**Topic 18:**  
monitor + computer + screen + power + vga + video + color + cable + switch + turn  

**Topic 19:**  
fbi + batf + koresh + child + compound + government + warrant + tank + davidian + waco  

**Topic 20:**  
mhz + dx + mb + motherboard + speed + cpu + ram + bus + board + run  

**Topic 21:**  
copy + protection + backup + software + question + pirate + machine + answer + user + make  

**Topic 22:**  
space + launch + shuttle + orbit + nasa + cost + satellite + moon + earth + station  

**Topic 23:**  
article + write + david + post + john + read + sure + mike + opinion + mark  

**Topic 24:**  
printer + print + laser + hp + deskjet + ink + bj + postscript + bubblejet + canon  

**Topic 25:**  
modem + port + serial + com + irq + bis + pin + pc + connect + internal  

**Topic 26:**  
jesus + christ + sin + heaven + law + christian + die + paul + father + bible  

**Topic 27:**  
problem + work + line + try + time + machine + fix + battery + error + change  

**Topic 28:**  
christian + church + religion + belief + faith + christianity + bible + believe + catholic + atheist  

**Topic 29:**  
scsi + ide + controller + bus + device + fast + mb + pc + adaptec + interface  

**Topic 30:**  
disease + doctor + msg + patient + effect + medical + food + study + test + gordon  

**Topic 31:**  
mac + apple + cd + lc + simms + rom + buy + duo + fpu + price  

**Topic 32:**  
hit + pitch + pitcher + year + run + clutch + ball + hitter + player + guy  

**Topic 33:**  
book + read + good + reference + chemistry + page + title + guide + publish + author  

**Topic 34:**  
image + color + bit + display + xv + screen + gif + jpeg + format + pixel  

**Topic 35:**  
chip + clipper + encryption + government + algorithm + crypto + nsa + privacy + secure + phone  

**Topic 36:**  
phone + university + internet + fax + computer + email + georgia + number + research + science  

**Topic 37:**  
objective + moral + morality + value + jon + keith + mean + goal + livesey + natural  

**Topic 38:**  
right + law + government + state + constitution + amendment + arm + court + federal + citizen  

**Topic 39:**  
motif + server + run + widget + application + display + sun + code + software + pc  

**Topic 40:**  
know + let + question + tell + mr + really + anybody + want + answer + appreciate  

**Topic 41:**  
program + graphic + icon + group + manager + ftp + format + shareware + package + object  

**Topic 42:**  
muslim + islam + serb + bosnian + muslims + war + islamic + bosnia + croat + world  

**Topic 43:**  
sale + offer + sell + price + condition + good + new + shipping + include + interested  

**Topic 44:**  
mouse + com + keyboard + windows + motion + microsoft + jump + sys + button + smoothly  

**Topic 45:**  
say + subject + thing + word + quote + fact + mean + previous + bob + speak  

**Topic 46:**  
disk + pc + hard + read + software + mb + boot + memory + floppy + manual  

**Topic 47:**  
font + truetype + atm + character + tt + change + size + small + type + default  

**Topic 48:**  
mail + list + post + address + send + reply + mailing + message + group + subscribe  

**Topic 49:**  
use + switch + device + audio + circuit + tape + signal + stuff + input + buy